# JetBot Camera Capture

This notebook captures 20 frames from the JetBot camera and stores them in a timestamped folder under `assets/`.
Run the cells in order. Re-run the second cell whenever you need a fresh capture set.

In [ ]:
import datetime as dt
from pathlib import Path
import time

from jetcam.usb_camera import USBCamera
from PIL import Image

ASSETS_DIR = Path('assets')
FRAME_COUNT = 20
FRAME_DELAY = 0.5  # seconds between captured frames
IMAGE_PREFIX = 'frame'
CAPTURE_SIZE = (224, 224)

ASSETS_DIR.mkdir(parents=True, exist_ok=True)
timestamp = dt.datetime.now().strftime('%Y%m%d_%H%M%S')
capture_dir = ASSETS_DIR / f'capture_{timestamp}'
capture_dir.mkdir()

print(f'Saving {FRAME_COUNT} frames to {capture_dir.resolve()}')

In [ ]:
# Start the camera, capture 20 frames, and show the live preview while saving.
# Re-run this cell if you need another batch; it will reuse the most recent capture_dir value.

import io
from IPython.display import display
import ipywidgets as widgets

camera = USBCamera(width=CAPTURE_SIZE[0], height=CAPTURE_SIZE[1])
preview_widget = widgets.Image(format='jpeg')
status_widget = widgets.HTML(value='<p>Waiting for frames…</p>')
preview_widget.layout.width = '320px'
preview_widget.layout.height = '240px'
display(widgets.VBox([preview_widget, status_widget]))

captured_paths = []
camera.running = True

try:
    for idx in range(FRAME_COUNT):
        frame = camera.value
        wait_count = 0
        while frame is None and wait_count < 50:
            time.sleep(0.1)
            frame = camera.value
            wait_count += 1
        if frame is None:
            raise RuntimeError('Camera did not provide a frame; check the connection and try again.')

        image = Image.fromarray(frame)
        buffer = io.BytesIO()
        image.save(buffer, format='JPEG')
        preview_widget.value = buffer.getvalue()

        image_path = capture_dir / f"{IMAGE_PREFIX}_{idx:02d}.jpg"
        image.save(image_path)
        captured_paths.append(image_path)
        status_widget.value = f'<p>Saved {image_path.name} ({idx + 1}/{FRAME_COUNT})</p>'
        time.sleep(FRAME_DELAY)
finally:
    camera.running = False
    cap = getattr(camera, 'cap', None)
    if cap is not None:
        cap.release()

thumb_widgets = []
for path in captured_paths:
    with path.open('rb') as f:
        thumb_widgets.append(widgets.Image(value=f.read(), format='jpeg', layout=widgets.Layout(width='160px')))

if thumb_widgets:
    display(widgets.HBox(thumb_widgets))
else:
    status_widget.value = '<p>No frames captured.</p>'

In [ ]:
from itertools import islice

saved_images = sorted(capture_dir.glob('*.jpg'))
print(f'{len(saved_images)} images saved in {capture_dir}:')
for path in islice(saved_images, 5):
    print('  ', path.name)

In [ ]:
# Detect chessboard corners in the latest capture folder and save annotated images.

import cv2 as cv
import numpy as np

pattern_size = (7, 6)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

capture_dirs = sorted((p for p in ASSETS_DIR.glob('capture_*') if p.is_dir()), key=lambda p: p.stat().st_mtime)
if not capture_dirs:
    raise RuntimeError('No capture directories found; run the capture cell first.')

source_dir = capture_dirs[-1]
timestamp = source_dir.name.replace('capture_', '', 1)
results_dir = ASSETS_DIR / f'capture_{timestamp}_chessboard'
results_dir.mkdir(exist_ok=True)

print(f'Processing images from {source_dir} -> {results_dir}')

objpoints = []
imgpoints = []

image_paths = sorted(source_dir.glob('*.jpg'))
for image_path in image_paths:
    img = cv.imread(str(image_path))
    if img is None:
        print(f'Skipping unreadable image: {image_path.name}')
        continue

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, pattern_size, None)

    if ret:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        annotated = img.copy()
        cv.drawChessboardCorners(annotated, pattern_size, corners2, ret)
        output_path = results_dir / image_path.name
        cv.imwrite(str(output_path), annotated)
        print(f'Detected chessboard: {image_path.name} -> {output_path.name}')
    else:
        print(f'No chessboard found in {image_path.name}')

print(f"Processed {len(image_paths)} images; detected chessboard in {len(imgpoints)} of them.")